In [6]:
import importlib
import prejokes
import pandas
from sklearn.model_selection import train_test_split 

In [33]:
jokes_file = '../data/s3_curated_jokes.txt'
nojoke_file = '../data/none_jokes.txt'

In [34]:
importlib.reload(prejokes)
jp = prejokes.JokePreprocessor()

## Load Data

In [35]:
joke_df = jp.load(jokes_file)
joke_df = jp.add_labels(joke_df, 1)
joke_df = joke_df[:200] # nur fur Testzwecke damit pos/neg halbwegsa ausgeglichen ist
nonjoke_df = jp.load(nojoke_file)
nonjoke_df = jp.add_labels(nonjoke_df, 0)
df = jp.merge_and_shuffle([joke_df, nonjoke_df])
# TODO: get max sequence length for padding later

In [36]:
MAX_SEQ_LEN = jp.get_max_seq_len(df, 'Joke')

## Data Preprocessing

### TEXT CLEANING

In [37]:
df = jp.remove_punctuation(df,'Joke')
df = jp.replace_numbers(df, 'Joke')
# TODO: contraction removal

### TEXT REPRESENTATION

In [38]:
data_train, data_test = train_test_split(df, test_size=0.10, random_state=42)
train_X = data_train['Joke'].tolist()
test_X = data_test['Joke'].tolist()

In [40]:
# Tokenizer
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000, lower=True, char_level=False)
tokenizer.fit_on_texts(train_X + test_X)

/Users/miriamamin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [42]:
# Sequencing
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [43]:
# Padding
from keras.preprocessing.sequence import pad_sequences
train_X = pad_sequences(train_X, maxlen=MAX_SEQ_LEN)
test_X = pad_sequences(test_X, maxlen=MAX_SEQ_LEN)

## Prepare Training Process